In [9]:
# 프로퍼티의 이해

class Natural:
    def __init__(self, n):
        if n < 0:
            self.__n = 0
        else:
            self.__n = n

    def getn(self):
        return self.__n
    
    def setn(self, n):
        if n < 0:
            self.__n = 0
        else:
            self.__n = n

    p = property(getn, setn) # property는 클래스이다. (클래스 변수)

n1 = Natural(1)

print(n1.getn())

n2 = Natural(-1)

# print(n2.getn())

print(n1.getn())
print(n1.p) # n1.getn() 이 호출되고 있다.

n1.setn(-20)
print(n1.getn())

n1.p = n2.p + n2.p

print(n1.getn())

1
1
1
0
0


In [1]:
class Citizen:
    def __init__(self, age):
        self._age = age
		
    @property # 프로퍼티 함수에 age 함수를 전달
    def age(self):
        return self._age
    
    @age.setter # age가 프로퍼티 객체가 되어서 안에 있는 setter 함수를 호출
    def age(self, age): # setter 함수에 들어갈 함수
        print("나이를 업데이트 합니다")
        self._age = age
	
citizen = Citizen(20)
citizen.age = 30
print(citizen.age)

나이를 업데이트 합니다
30


In [2]:
# 프로퍼티 클래스의 원리

class CharacterInfo: # getter, setter, delete 를 하나라도 가지고 있는 클래스를 디스크립터 클래스라 부른다.
    def __init__(self, power, speed):
        self.power = power
        self.speed = speed

    def __get__(self, obj, objtype): # 혼자 있을때 호출되는 함수
        print('(GET)정보 조회됨')
        # <__main__.CharacterInfo object at 0x000001BCB2AD7D30> 
        # <__main__.Guardian object at 0x000001BCB2AD7BB0> 
        # <class '__main__.Guardian'>
        print(self, obj, objtype)
        return ("공격력 : "+str(self.power) + " / 스피드 : " + self.speed)

    def __set__(self, obj, val): # = 왼쪽에 있으면 실행되는 함수
        print('(UPDATE)정보 갱신 시작')
        self.power = val.power
        self.speed = val.speed

    def __delete__(self, obj):
        print("(DELETE)정보 삭제하기")
        self.power =""
        self.speed = ""

class Guardian:
    info = CharacterInfo(10, "50km/h")


g1 = Guardian()   # g1 이라는 수호천사 인스턴스 하나 생성
print(g1.info)   # 인스턴스 g1의 초기 정보 출력. __get__ 실행.
info_after_upgrade = CharacterInfo(15, "70km/h")   # 업그레이드 아이템 적용 후 캐릭터 정보
g1.info = info_after_upgrade   # 새 캐릭터 정보를 인스턴스 g1 에 설정. = 왼쪽에 있어서 __set__ 실행
print(g1.info)   # 인스턴스 g1의 정보 출력
del g1.info   # 인스턴스 g1의 정보 삭제
print(g1.info)   # 인스턴스 g1의 정보 출력

(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001D78861E230> <__main__.Guardian object at 0x000001D78861D660> <class '__main__.Guardian'>
공격력 : 10 / 스피드 : 50km/h
(UPDATE)정보 갱신 시작
(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001D78861E230> <__main__.Guardian object at 0x000001D78861D660> <class '__main__.Guardian'>
공격력 : 15 / 스피드 : 70km/h
(DELETE)정보 삭제하기
(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001D78861E230> <__main__.Guardian object at 0x000001D78861D660> <class '__main__.Guardian'>
공격력 :  / 스피드 : 


In [3]:
class Car:
    def __init__(self, initial_speed):
        self._speed = initial_speed

    @property # __get__
    def speed(self):
        """I'm the 'speed' property."""
        print('현재 속도 구하기')
        return self._speed

    @speed.setter # __set__
    def speed(self, value):
        print('현재 속도 설정하기')
        self._speed = value

    @speed.deleter
    def speed(self):
        print('현재 속도 정보 삭제하기')
        del self._speed


car = Car(50)
print(car.speed)
car.speed = 100
print(car.speed)
del car.speed



현재 속도 구하기
50
현재 속도 설정하기
현재 속도 구하기
100
현재 속도 정보 삭제하기


In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __set_name__(self, owner, name):
        self.__name__ = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)